In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("Sales Analysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/02 16:39:11 WARN Utils: Your hostname, Aarons-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.111 instead (on interface en0)
25/06/02 16:39:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/02 16:39:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:

# Read the CSV file
df = spark.read.csv('../../data/Sales Dataset.csv', header=True, inferSchema=True)

print("Schema of the DataFrame:")
df.printSchema()
print("Sample data:")
df.show()

Schema of the DataFrame:
root
 |-- _c0: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price per Unit: integer (nullable = true)
 |-- Total Amount: integer (nullable = true)

Sample data:
+---+----------+------+---+----------------+--------+--------------+------------+
|_c0|      Date|Gender|Age|Product Category|Quantity|Price per Unit|Total Amount|
+---+----------+------+---+----------------+--------+--------------+------------+
|  0|2023-11-24|  Male| 34|          Beauty|       3|            50|         150|
|  1|2023-02-27|Female| 26|        Clothing|       2|           500|        1000|
|  2|2023-01-13|  Male| 50|     Electronics|       1|            30|          30|
|  3|2023-05-21|  Male| 37|        Clothing|       1|           500|         500|
|  4|2023-05-06|  Male| 30|          Beauty|       2|    

25/06/02 16:40:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Gender, Age, Product Category, Quantity, Price per Unit, Total Amount
 Schema: _c0, Date, Gender, Age, Product Category, Quantity, Price per Unit, Total Amount
Expected: _c0 but found: 
CSV file: file:///Users/aaronginder/Git/event-store/data/Sales%20Dataset.csv


In [ ]:
# Cleaning
# Drop the first index column
df = df.drop('_c0')
df.show()

In [10]:
df.show()
df.describe().show()

+----------+------+---+----------------+--------+--------------+------------+
|      Date|Gender|Age|Product Category|Quantity|Price per Unit|Total Amount|
+----------+------+---+----------------+--------+--------------+------------+
|2023-11-24|  Male| 34|          Beauty|       3|            50|         150|
|2023-02-27|Female| 26|        Clothing|       2|           500|        1000|
|2023-01-13|  Male| 50|     Electronics|       1|            30|          30|
|2023-05-21|  Male| 37|        Clothing|       1|           500|         500|
|2023-05-06|  Male| 30|          Beauty|       2|            50|         100|
|2023-04-25|Female| 45|          Beauty|       1|            30|          30|
|2023-03-13|  Male| 46|        Clothing|       2|            25|          50|
|2023-02-22|  Male| 30|     Electronics|       4|            25|         100|
|2023-12-13|  Male| 63|     Electronics|       2|           300|         600|
|2023-10-07|Female| 52|        Clothing|       4|            50|

25/06/02 16:42:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------+------------------+----------------+------------------+------------------+-----------------+
|summary|Gender|               Age|Product Category|          Quantity|    Price per Unit|     Total Amount|
+-------+------+------------------+----------------+------------------+------------------+-----------------+
|  count|  1000|              1000|            1000|              1000|              1000|             1000|
|   mean|  NULL|            41.392|            NULL|             2.514|            179.89|            456.0|
| stddev|  NULL|13.681429659122518|            NULL|1.1327343409145354|189.68135627129234|559.9976315551235|
|    min|Female|                18|          Beauty|                 1|                25|               25|
|    max|  Male|                64|     Electronics|                 4|               500|             2000|
+-------+------+------------------+----------------+------------------+------------------+-----------------+



In [11]:
from pyspark.sql.functions import col, date_format

df = df.withColumn("year", date_format(col("Date"), "yyyy")) \
    .withColumn("month", date_format(col("Date"), "MM")) \
    .withColumn("day", date_format(col("Date"), "dd"))

In [13]:
df.write.mode("overwrite").partitionBy("year", "month", "day")\
    .parquet("../data/")

ConnectionRefusedError: [Errno 61] Connection refused